In [1]:
import os
import csv
from urlparse import urlparse
import boto3
import numpy as np
from scipy.sparse import lil_matrix

BUCKET = 'proserve-emea-ai-workshops'
s3 = boto3.client('s3')


def download_file(s3_source, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)        

    url = urlparse(s3_source)
    bucket, key = url.netloc, url.path.lstrip('/')
    file_name = key.split('/')[-1]
    with open('%s/%s' % (dest,file_name), 'wb') as data:
      s3.download_fileobj(bucket, key, data)


def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            Y.append(int(rating))
            line=line+1

    Y=np.array(Y).astype('float32')
    return X,Y

nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies
nbRatingsTrain=80000
nbRatingsTest=20000

input_dir = 'input/100k'
download_file('s3://%s/recommenders/workshop-artifacts/data-inputs/movielens100k/u1.base' % BUCKET, input_dir)
download_file('s3://%s/recommenders/workshop-artifacts/data-inputs/movielens100k/u1.test' % BUCKET, input_dir)

X_train, Y_train = loadDataset('%s/u1.base' % input_dir, nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('%s/u1.test' % input_dir, nbRatingsTest, nbFeatures)

In [2]:
prefix = 'exercise4/fm-movielens100k'
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')
test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')
output_prefix  = 's3://{}/{}/output'.format(BUCKET, prefix)

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    import io,boto3
    import sagemaker.amazon.common as smac
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    print buf
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    print('Wrote dataset: {}/{}'.format(bucket,obj))
    
writeDatasetToProtobuf(X_train, Y_train, BUCKET, train_prefix, train_key)    
writeDatasetToProtobuf(X_test, Y_test, BUCKET, test_prefix, test_key)    
  
print('Output: {}'.format(output_prefix))

Wrote dataset: proserve-emea-ai-workshops/exercise4/fm-movielens100k/train/train.protobuf
Wrote dataset: proserve-emea-ai-workshops/exercise4/fm-movielens100k/test/test.protobuf
Output: s3://proserve-emea-ai-workshops/exercise4/fm-movielens100k/output


In [3]:
import sagemaker
from sagemaker import get_execution_role

train_data = 's3://%s/exercise4/fm-movielens100k/train/train.protobuf' % BUCKET
test_data = 's3://%s/exercise4/fm-movielens100k/test/test.protobuf' % BUCKET

containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='regressor',
                      mini_batch_size=1000,
                      num_factors=64,
                      _speedometer_period=10,
                      epochs=50)

fm.fit({'train': train_data, 'test': test_data})

INFO:sagemaker:Creating training-job with name: factorization-machines-2018-03-21-23-40-14-697


......................................................................
Docker entrypoint called with argument(s): train
[03/21/2018 23:45:57 INFO 139926754449216] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'bias_wd': u'0.01', u'use_linear': u'true', u'_speedometer_period': u'500', u'bias_lr': u'0.1', u'mini_batch_size': u'1000', u'_use_full_symbolic': u'true', u'bias_init_sigma': u'0.01', u'_num_gpus': u'auto', u'_data_format': u'record', u'factors_wd': u'0.00001', u'linear_wd': u'0.001', u'_kvstore': u'auto', u'_learning_rate': u'1.0', u'_optimizer': u'adam'}
[03/21/2018 23:45:57 INFO 13992675444921

[03/21/2018 23:46:01 INFO 139926754449216] Epoch[1] Batch [10]#011Speed: 187188.89 samples/sec#011rmse=1.116834
[03/21/2018 23:46:01 INFO 139926754449216] Epoch[1] Batch [20]#011Speed: 150963.85 samples/sec#011rmse=1.084974
[03/21/2018 23:46:01 INFO 139926754449216] Epoch[1] Batch [30]#011Speed: 176246.07 samples/sec#011rmse=1.098281
[03/21/2018 23:46:02 INFO 139926754449216] Epoch[1] Batch [40]#011Speed: 170954.65 samples/sec#011rmse=1.085589
[03/21/2018 23:46:02 INFO 139926754449216] Epoch[1] Batch [50]#011Speed: 145577.43 samples/sec#011rmse=1.079706
[03/21/2018 23:46:02 INFO 139926754449216] Epoch[1] Batch [60]#011Speed: 181412.18 samples/sec#011rmse=1.073165
[03/21/2018 23:46:02 INFO 139926754449216] Epoch[1] Batch [70]#011Speed: 145165.28 samples/sec#011rmse=1.070668
#metrics {"Metrics": {"update.time": {"count": 1, "max": 489.66002464294434, "sum": 489.66002464294434, "min": 489.66002464294434}}, "EndTime": 1521675962.253004, "Dimensions": {"Host": "algo-1", "Operation": "traini

#metrics {"Metrics": {"update.time": {"count": 1, "max": 514.1129493713379, "sum": 514.1129493713379, "min": 514.1129493713379}}, "EndTime": 1521675966.824455, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1521675966.310086}

#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Batches Since Last Reset": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Records Since Last Reset": {"count": 1, "max": 80000, "sum": 80000.0, "min": 80000}, "Total Batches Seen": {"count": 1, "max": 1361, "sum": 1361.0, "min": 1361}, "Total Records Seen": {"count": 1, "max": 1361000, "sum": 1361000.0, "min": 1361000}, "Max Records Seen Between Resets": {"count": 1, "max": 80000, "sum": 80000.0, "min": 80000}, "Reset Count": {"count": 1, "max": 18, "sum": 18.0, "min": 18}}, "EndTime": 1521675966.824627, "Dimensions": {"Host": "algo-1", "Meta": "training_data_iter", "Opera

[03/21/2018 23:46:11 INFO 139926754449216] Epoch[1] Batch [50]#011Speed: 149979.22 samples/sec#011rmse=1.003584
[03/21/2018 23:46:11 INFO 139926754449216] Epoch[1] Batch [60]#011Speed: 150849.83 samples/sec#011rmse=0.996394
[03/21/2018 23:46:11 INFO 139926754449216] Epoch[1] Batch [70]#011Speed: 151678.47 samples/sec#011rmse=0.993622
#metrics {"Metrics": {"update.time": {"count": 1, "max": 535.1331233978271, "sum": 535.1331233978271, "min": 535.1331233978271}}, "EndTime": 1521675971.998793, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1521675971.463378}

#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Batches Since Last Reset": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Records Since Last Reset": {"count": 1, "max": 80000, "sum": 80000.0, "min": 80000}, "Total Batches Seen": {"count": 1, "max": 2161, "sum": 2161.0, "min": 2161}, "Total 

[03/21/2018 23:46:16 INFO 139926754449216] Epoch[1] Batch [30]#011Speed: 148467.26 samples/sec#011rmse=0.991802
[03/21/2018 23:46:16 INFO 139926754449216] Epoch[1] Batch [40]#011Speed: 145875.14 samples/sec#011rmse=0.981943
[03/21/2018 23:46:16 INFO 139926754449216] Epoch[1] Batch [50]#011Speed: 186036.48 samples/sec#011rmse=0.979413
[03/21/2018 23:46:17 INFO 139926754449216] Epoch[1] Batch [60]#011Speed: 177920.01 samples/sec#011rmse=0.972314
[03/21/2018 23:46:17 INFO 139926754449216] Epoch[1] Batch [70]#011Speed: 156521.40 samples/sec#011rmse=0.969504
#metrics {"Metrics": {"update.time": {"count": 1, "max": 513.4570598602295, "sum": 513.4570598602295, "min": 513.4570598602295}}, "EndTime": 1521675977.155793, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1521675976.642091}

#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Batches Since Last Reset": {"cou

[03/21/2018 23:46:21 INFO 139926754449216] Epoch[1] Batch [70]#011Speed: 144264.54 samples/sec#011rmse=0.952254
#metrics {"Metrics": {"update.time": {"count": 1, "max": 521.5380191802979, "sum": 521.5380191802979, "min": 521.5380191802979}}, "EndTime": 1521675981.871628, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1521675981.349865}

#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Batches Since Last Reset": {"count": 1, "max": 80, "sum": 80.0, "min": 80}, "Number of Records Since Last Reset": {"count": 1, "max": 80000, "sum": 80000.0, "min": 80000}, "Total Batches Seen": {"count": 1, "max": 3681, "sum": 3681.0, "min": 3681}, "Total Records Seen": {"count": 1, "max": 3681000, "sum": 3681000.0, "min": 3681000}, "Max Records Seen Between Resets": {"count": 1, "max": 80000, "sum": 80000.0, "min": 80000}, "Reset Count": {"count": 1, "max": 47, "sum": 47.0, 

===== Job Complete =====
Billable seconds: 217


In [4]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)


INFO:sagemaker:Creating model with name: factorization-machines-2018-03-21-23-47-57-115
INFO:sagemaker:Creating endpoint with name factorization-machines-2018-03-21-23-40-14-697


---------------------------------------------------------------------------------------------------!

In [5]:
import json
import numpy as np
from sagemaker.predictor import json_deserializer

nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        keys = np.argwhere(row == np.amax(row)).flatten().tolist()
        js['instances'].append({
            'data':{
                'features': {
                    'keys': keys,
                    'shape': [nbFeatures],
                    'values': [1]*len(keys)
                }
            }
            })
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer


result = fm_predictor.predict(X_test[1000:1010].toarray())
print(result)
print()
print (Y_test[1000:1010])

{u'predictions': [{u'score': 3.3320837020874023}, {u'score': 3.0627427101135254}, {u'score': 3.305492639541626}, {u'score': 2.9380016326904297}, {u'score': 2.8458235263824463}, {u'score': 3.073624849319458}, {u'score': 3.040721893310547}, {u'score': 3.3230855464935303}, {u'score': 3.044969081878662}, {u'score': 3.535712480545044}]}
()
[2. 1. 3. 3. 3. 1. 3. 3. 1. 4.]
